In [ ]:
!pip install category_encoders
from category_encoders import TargetEncoder
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import linear_model
from plotly import graph_objs, figure_factory
import statsmodels.api as sm

CO2 = pd.read_csv('/content/drive/MyDrive/CO2 Emissions_Canada.csv')
CO2.info()

In [ ]:
print(CO2['Model'].nunique())
CO2_1 = CO2.drop('Model', axis=1)

2053


In [ ]:
X_TE = CO2_1.drop(['CO2 Emissions(g/km)'], axis = 1)
Y = CO2_1['CO2 Emissions(g/km)']
print(X_TE.shape)
print(Y.shape)

(7385, 10)
(7385,)


In [ ]:
print(X_TE['Make'].nunique())
print(X_TE['Transmission'].nunique())
print(X_TE['Vehicle Class'].nunique())
Target_Encoder = TargetEncoder(cols=['Make', 'Transmission', 'Vehicle Class']).fit(X_TE,Y)
X_TE = Target_Encoder.transform(X_TE)
print(X_TE['Make'].nunique())
print(X_TE['Transmission'].nunique())
print(X_TE['Vehicle Class'].nunique())

42
27
16
42
27
16


In [ ]:
X_TE2 = pd.get_dummies(X_TE, ['Fuel Type'], dtype=int)
X_TE2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Make                              7385 non-null   float64
 1   Vehicle Class                     7385 non-null   float64
 2   Engine Size(L)                    7385 non-null   float64
 3   Cylinders                         7385 non-null   int64  
 4   Transmission                      7385 non-null   float64
 5   Fuel Consumption City (L/100 km)  7385 non-null   float64
 6   Fuel Consumption Hwy (L/100 km)   7385 non-null   float64
 7   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 8   Fuel Consumption Comb (mpg)       7385 non-null   int64  
 9   Fuel Type_D                       7385 non-null   int64  
 10  Fuel Type_E                       7385 non-null   int64  
 11  Fuel Type_N                       7385 non-null   int64  
 12  Fuel T

In [ ]:
correlation  = X_TE2.corr()
f = figure_factory.create_annotated_heatmap(correlation.values,list(correlation.columns),list(correlation.columns),correlation.round(2).values,showscale=True)
f.show()

In [ ]:
X_TE2 = X_TE2.drop(['Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)'], axis=1)
X_TE2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Make                              7385 non-null   float64
 1   Vehicle Class                     7385 non-null   float64
 2   Engine Size(L)                    7385 non-null   float64
 3   Cylinders                         7385 non-null   int64  
 4   Transmission                      7385 non-null   float64
 5   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 6   Fuel Consumption Comb (mpg)       7385 non-null   int64  
 7   Fuel Type_D                       7385 non-null   int64  
 8   Fuel Type_E                       7385 non-null   int64  
 9   Fuel Type_N                       7385 non-null   int64  
 10  Fuel Type_X                       7385 non-null   int64  
 11  Fuel Type_Z                       7385 non-null   int64  
dtypes: flo

In [ ]:
print(X_TE2.shape)
print(Y.shape)

(7385, 12)
(7385,)


In [ ]:
X_Scaled= StandardScaler().fit_transform(X_TE2)
pd.DataFrame(X_Scaled)

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.776306,-0.985321,-0.856721,-0.883408,0.529076,-0.855742,0.763110,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
1,-0.776306,-0.985321,-0.561317,-0.883408,-0.617314,-0.475423,0.209966,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
2,-0.776306,-0.985321,-1.225976,-0.883408,-1.928696,-1.754677,2.837400,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
3,-0.776306,-0.412662,0.251043,0.210575,-0.348953,0.043193,-0.343178,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
4,-0.776306,-0.412662,0.251043,0.210575,-0.348953,-0.129679,-0.066606,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
...,...,...,...,...,...,...,...,...,...,...,...,...
7380,-0.593434,-0.412662,-0.856721,-0.883408,0.420931,-0.544572,0.348252,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
7381,-0.593434,-0.412662,-0.856721,-0.883408,0.420931,-0.371700,0.209966,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
7382,-0.593434,-0.412662,-0.856721,-0.883408,0.420931,-0.233402,-0.066606,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
7383,-0.593434,1.588643,-0.856721,-0.883408,0.420931,-0.371700,0.209966,-0.155794,-0.229661,-0.011637,-0.985081,1.142966


In [ ]:
X_Scaled = sm.add_constant(X_Scaled)

In [ ]:
X_Scaled_df = pd.DataFrame(X_Scaled, columns=['const'] + list(X_TE2.columns))

In [ ]:
model = sm.OLS(Y, X_Scaled_df)
results = model.fit()
print(results.summary())

                             OLS Regression Results                            
Dep. Variable:     CO2 Emissions(g/km)   R-squared:                       0.993
Model:                             OLS   Adj. R-squared:                  0.993
Method:                  Least Squares   F-statistic:                 9.321e+04
Date:                 Sun, 28 Jul 2024   Prob (F-statistic):               0.00
Time:                         10:25:48   Log-Likelihood:                -22281.
No. Observations:                 7385   AIC:                         4.459e+04
Df Residuals:                     7373   BIC:                         4.467e+04
Df Model:                           11                                         
Covariance Type:             nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [ ]:
p_values = results.pvalues
print("P-values:\n", p_values)

P-values:
 const                                0.000000e+00
Make                                 5.534735e-24
Vehicle Class                        2.474346e-04
Engine Size(L)                       8.012723e-03
Cylinders                            9.036160e-24
Transmission                         3.450380e-07
Fuel Consumption Comb (L/100 km)     0.000000e+00
Fuel Consumption Comb (mpg)         6.707665e-277
Fuel Type_D                          0.000000e+00
Fuel Type_E                          0.000000e+00
Fuel Type_N                          5.284147e-50
Fuel Type_X                          0.000000e+00
Fuel Type_Z                          0.000000e+00
dtype: float64


In [ ]:
columns_to_keep = p_values[p_values <= 0.05].index
df_filtered = X_Scaled_df[columns_to_keep]
print("Filtered DataFrame:\n", df_filtered)

Filtered DataFrame:
       const      Make  Vehicle Class  Engine Size(L)  Cylinders  Transmission  \
0       1.0 -0.776306      -0.985321       -0.856721  -0.883408      0.529076   
1       1.0 -0.776306      -0.985321       -0.561317  -0.883408     -0.617314   
2       1.0 -0.776306      -0.985321       -1.225976  -0.883408     -1.928696   
3       1.0 -0.776306      -0.412662        0.251043   0.210575     -0.348953   
4       1.0 -0.776306      -0.412662        0.251043   0.210575     -0.348953   
...     ...       ...            ...             ...        ...           ...   
7380    1.0 -0.593434      -0.412662       -0.856721  -0.883408      0.420931   
7381    1.0 -0.593434      -0.412662       -0.856721  -0.883408      0.420931   
7382    1.0 -0.593434      -0.412662       -0.856721  -0.883408      0.420931   
7383    1.0 -0.593434       1.588643       -0.856721  -0.883408      0.420931   
7384    1.0 -0.593434       1.588643       -0.856721  -0.883408      0.420931   

      

In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   const                             7385 non-null   float64
 1   Make                              7385 non-null   float64
 2   Vehicle Class                     7385 non-null   float64
 3   Engine Size(L)                    7385 non-null   float64
 4   Cylinders                         7385 non-null   float64
 5   Transmission                      7385 non-null   float64
 6   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 7   Fuel Consumption Comb (mpg)       7385 non-null   float64
 8   Fuel Type_D                       7385 non-null   float64
 9   Fuel Type_E                       7385 non-null   float64
 10  Fuel Type_N                       7385 non-null   float64
 11  Fuel Type_X                       7385 non-null   float64
 12  Fuel T

In [ ]:
df_filtered.tail(20)

,const,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
7365,1.0,-1.199117,-0.985321,-0.856721,-0.883408,-0.617314,-0.786593,0.624824,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7366,1.0,-1.199117,-0.985321,-1.299827,-0.883408,0.420931,-1.374358,1.731112,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7367,1.0,-1.199117,-0.985321,-1.299827,-0.883408,-0.617314,-1.374358,1.731112,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7368,1.0,-1.199117,-0.985321,-0.856721,-0.883408,0.591066,-0.890316,0.901396,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7369,1.0,-1.199117,-0.985321,-0.856721,-0.883408,-0.617314,-0.855742,0.763110,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7370,1.0,-1.199117,-0.816667,-0.856721,-0.883408,-0.348953,-0.786593,0.624824,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7371,1.0,-1.199117,-0.412662,-0.856721,-0.883408,0.420931,-0.544572,0.348252,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7372,1.0,-1.199117,-0.412662,-0.856721,-0.883408,0.420931,-0.267976,0.071680,-0.155794,-0.229661,-0.011637,1.015145,-0.874917
7373,1.0,-0.593434,-0.985321,-0.856721,-0.883408,0.420931,-0.717444,0.624824,-0.155794,-0.229661,-0.011637,-0.985081,1.142966
7374,1.0,-0.593434,-0.985321,-0.856721,-0.883408,0.420931,-0.544572,0.348252,-0.155794,-0.229661,-0.011637,-0.985081,1.142966


In [ ]:
SVRegressorC = SVR()
HparameterSVR1 = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [200, 400, 500, 600, 700], 'epsilon': [10, 20, 30, 50, 100]}
grid_search1 = GridSearchCV(estimator=SVRegressorC, param_grid=HparameterSVR1, scoring='r2', cv=5)
grid_search1.fit(df_filtered, Y)
SVR_C_best_parameters = grid_search1.best_params_
print("Best parameters: ", SVR_C_best_parameters)
SVR_C_best_result = grid_search1.best_score_
print("Best result: ", SVR_C_best_result)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-SVR_C_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)

Best parameters:  {'C': 400, 'epsilon': 10, 'kernel': 'rbf'}
Best result:  0.9927138002751617
(7385, 13)
modified_r2:  0.992697729593719


In [ ]:
SVRegressor = SVR()
HparameterSVR2 = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'epsilon': [10, 20, 30, 50, 100]}
grid_search2 = GridSearchCV(estimator=SVRegressor, param_grid=HparameterSVR2, scoring='r2', cv=5)
grid_search2.fit(df_filtered, Y)
SVR_best_parameters = grid_search2.best_params_
print("Best parameters: ", SVR_best_parameters)
SVR_best_result = grid_search2.best_score_
print("Best result: ", SVR_best_result)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-SVR_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)

Best parameters:  {'epsilon': 10, 'kernel': 'linear'}
Best result:  0.9887932070320516
(7385, 13)
modified_r2:  0.9887684889613724


In [ ]:
LinearRegression = linear_model.SGDRegressor(random_state = 1, penalty = None)
HparameterLR = {'eta0': [.0001, .005, .001, .05, .01, .1, 1], 'max_iter':[100, 200, 300, 400, 500]}
grid_search3 = GridSearchCV(estimator=LinearRegression, param_grid=HparameterLR, scoring='r2', cv=5)
grid_search3.fit(df_filtered, Y)
LR1_best_parameters = grid_search3.best_params_
print("Best parameters: ", LR1_best_parameters)
LR1_best_result = grid_search3.best_score_
print("Best result: ", LR1_best_result)
LR1_best_model = grid_search3.best_estimator_
print("Intercept β0: ", LR1_best_model.intercept_)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-LR1_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)
print(pd.DataFrame(zip(df_filtered.columns, LR1_best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'eta0': 0.001, 'max_iter': 300}
Best result:  0.9912176130742069
Intercept β0:  [125.29201067]
(7385, 13)
modified_r2:  0.9911982423531286
                            Features  Coefficients
0                              const    125.292011
6   Fuel Consumption Comb (L/100 km)     55.130233
8                        Fuel Type_D      5.714986
11                       Fuel Type_X      3.976848
12                       Fuel Type_Z      3.636513
4                          Cylinders      2.079228
1                               Make      0.825648
3                     Engine Size(L)      0.621884
2                      Vehicle Class      0.480501
5                       Transmission     -0.502466
10                       Fuel Type_N     -0.861377
7        Fuel Consumption Comb (mpg)     -7.774563
9                        Fuel Type_E    -21.313257


In [ ]:
LinearRegressionl1 = linear_model.SGDRegressor(random_state = 1, penalty = 'l1')
HparameterLR = {'eta0': [.0001, .005, .001, .05, .01, .1, 1], 'max_iter':[50, 100, 150, 200, 300],'alpha': [10, 50, 100, 200, 500]}
grid_search4 = GridSearchCV(estimator=LinearRegressionl1, param_grid=HparameterLR, scoring='r2', cv=5)
grid_search4.fit(df_filtered, Y)
LR2_best_parameters = grid_search4.best_params_
print("Best parameters: ", LR2_best_parameters)
LR2_best_result = grid_search4.best_score_
print("Best result: ", LR2_best_result)
LR2_best_model = grid_search4.best_estimator_
print("Intercept β0: ", LR2_best_model.intercept_)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-LR2_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)
print(pd.DataFrame(zip(df_filtered.columns, LR2_best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'alpha': 10, 'eta0': 0.05, 'max_iter': 50}
Best result:  0.8739002825152337
Intercept β0:  [249.53143191]
(7385, 13)
modified_r2:  0.8736221528363565
                            Features  Coefficients
6   Fuel Consumption Comb (L/100 km)     16.432298
4                          Cylinders      8.259412
3                     Engine Size(L)      6.104568
0                              const      0.000000
1                               Make      0.000000
2                      Vehicle Class      0.000000
5                       Transmission      0.000000
8                        Fuel Type_D      0.000000
9                        Fuel Type_E      0.000000
10                       Fuel Type_N      0.000000
11                       Fuel Type_X      0.000000
12                       Fuel Type_Z      0.000000
7        Fuel Consumption Comb (mpg)    -17.724729


In [ ]:
LinearRegressionl2 = linear_model.SGDRegressor(random_state = 1, penalty = 'l2')
HparameterLR = {'eta0': [.0001, .005, .001, .05, .01, .1, 1], 'max_iter':[50, 100, 150, 200, 300],'alpha': [10, 50, 100, 200, 500]}
grid_search5 = GridSearchCV(estimator=LinearRegressionl2, param_grid=HparameterLR, scoring='r2', cv=5)
grid_search5.fit(df_filtered, Y)
LR3_best_parameters = grid_search5.best_params_
print("Best parameters: ", LR3_best_parameters)
LR3_best_result = grid_search5.best_score_
print("Best result: ", LR3_best_result)
LR3_best_model = grid_search5.best_estimator_
print("Intercept β0: ", LR3_best_model.intercept_)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-LR3_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)
print(pd.DataFrame(zip(df_filtered.columns, LR3_best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'alpha': 10, 'eta0': 0.1, 'max_iter': 50}
Best result:  0.5174562267313126
Intercept β0:  [251.48694259]
(7385, 13)
modified_r2:  0.5163919123348938
                            Features  Coefficients
3                     Engine Size(L)      4.370461
4                          Cylinders      4.367124
6   Fuel Consumption Comb (L/100 km)      4.029237
5                       Transmission      3.703102
2                      Vehicle Class      2.615918
1                               Make      1.977992
12                       Fuel Type_Z      1.236616
0                              const      1.022833
8                        Fuel Type_D      0.189049
10                       Fuel Type_N     -0.011903
11                       Fuel Type_X     -0.859108
9                        Fuel Type_E     -0.971509
7        Fuel Consumption Comb (mpg)     -3.371510


In [ ]:
LinearRegressionEN = linear_model.SGDRegressor(random_state = 1, penalty = 'elasticnet')
HparameterLR2 = {'eta0': [.0001, .005, .001, .05, .01, .1, 1], 'max_iter':[50, 100, 150, 200, 300],'alpha': [10, 50, 100, 200, 500], 'l1_ratio': [0.1, 0.2, 0.5, 0.8, 0.9]}
grid_search6 = GridSearchCV(estimator=LinearRegressionEN, param_grid=HparameterLR2, scoring='r2', cv=5)
grid_search6.fit(df_filtered, Y)
LR4_best_parameters = grid_search6.best_params_
print("Best parameters: ", LR4_best_parameters)
LR4_best_result = grid_search6.best_score_
print("Best result: ", LR4_best_result)
LR4_best_model = grid_search6.best_estimator_
print("Intercept β0: ", LR4_best_model.intercept_)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-LR4_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)
print(pd.DataFrame(zip(df_filtered.columns, LR4_best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'alpha': 10, 'eta0': 0.01, 'l1_ratio': 0.9, 'max_iter': 50}
Best result:  0.8091275298640636
Intercept β0:  [250.6252478]
(7385, 13)
modified_r2:  0.8087065352743508
                            Features  Coefficients
6   Fuel Consumption Comb (L/100 km)      9.955548
3                     Engine Size(L)      7.604288
4                          Cylinders      7.466789
1                               Make      2.703897
2                      Vehicle Class      1.934711
5                       Transmission      1.509099
0                              const      0.000000
8                        Fuel Type_D      0.000000
9                        Fuel Type_E      0.000000
10                       Fuel Type_N      0.000000
11                       Fuel Type_X      0.000000
12                       Fuel Type_Z      0.000000
7        Fuel Consumption Comb (mpg)    -10.366137


In [ ]:
RF_Regressor1 = RandomForestRegressor(criterion='squared_error', random_state=1)
HparameterRF = {'n_estimators': [25, 50, 75, 100, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [50, 100, 150, 200, 500]}
grid_search7 = GridSearchCV(estimator=RF_Regressor1, param_grid=HparameterRF, scoring='r2', cv=5)
grid_search7.fit(df_filtered, Y)
RF1_best_parameters = grid_search7.best_params_
print("Best parameters: ", RF1_best_parameters)
RF1_best_result = grid_search7.best_score_
print("best_score: ", RF1_best_result)
r, c=df_filtered.shape
print(df_filtered.shape)
modified_r2 = 1-(1-RF1_best_result)*(4/5*r-1)/(4/5*r-c-1)
print("modified_r2: ", modified_r2)
Important_feature1 = pd.Series(grid_search7.best_estimator_.feature_importances_, index=list(df_filtered)).sort_values(ascending=False)
print(Important_feature1)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning:


125 fits failed out of a total of 375.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

Best parameters:  {'max_depth': 50, 'max_features': 'sqrt', 'n_estimators': 50}
best_score:  0.9949361379620978
(7385, 13)
modified_r2:  0.9949249689416545
Fuel Consumption Comb (L/100 km)    0.331451
Fuel Consumption Comb (mpg)         0.274665
Engine Size(L)                      0.130226
Cylinders                           0.112837
Vehicle Class                       0.060009
Make                                0.033704
Transmission                        0.023993
Fuel Type_E                         0.015018
Fuel Type_X                         0.009084
Fuel Type_Z                         0.006794
Fuel Type_D                         0.002076
Fuel Type_N                         0.000142
const                               0.000000
dtype: float64


In [ ]:
best_model = grid_search7.best_estimator_

In [ ]:
import joblib
joblib.dump(best_model, "model.pkl")

['model.pkl']

In [ ]:
My_model = joblib.load("model.pkl")
x=[[1.0,	-1.199117	,	-0.985321,	-1.225976,	-0.883408,	-1.928696,	-1.754677,	2.837400,	-0.155794,	-0.229661,	-0.011637,	-0.985081,	-0.874917]]
predict=My_model.predict(x)[0]
predict=(round(My_model.predict(x)[0],2))
print(predict)

144.33


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

